In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('psm20-21data.txt')
winter2 = pd.read_csv ('psm21-22data.txt')
winter3 = pd.read_csv ('psm22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 55]

In [8]:
precip_days1.dtypes


station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
psm = LocationInfo(
    'Portsmouth', 'New Hampshire', 
    latitude=43.0718,  
    longitude=-70.7626,
    timezone='America/New_Hampshire'
)

psm

LocationInfo(name='Portsmouth', region='New Hampshire', timezone='America/New_Hampshire', latitude=43.0718, longitude=-70.7626)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(psm.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(psm.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(psm.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:36  2020-12-01 16:08:47  2020-12-01
1   2020-12-02 06:56:40  2020-12-02 16:08:31  2020-12-02
2   2020-12-03 06:57:42  2020-12-03 16:08:17  2020-12-03
3   2020-12-04 06:58:44  2020-12-04 16:08:05  2020-12-04
4   2020-12-05 06:59:44  2020-12-05 16:07:56  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:26:57  2021-02-24 17:26:03  2021-02-24
86  2021-02-25 06:25:21  2021-02-25 17:27:20  2021-02-25
87  2021-02-26 06:23:45  2021-02-26 17:28:36  2021-02-26
88  2021-02-27 06:22:08  2021-02-27 17:29:52  2021-02-27
89  2021-02-28 06:20:30  2021-02-28 17:31:07  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:20  2021-12-01 16:08:51  2021-12-01
1   2021-12-02 06:56:24  2021-12-02 16:08:34  2021-12-02
2   2021-12-03 06:57:27  2021-12-03 16:08:20  2021-12-03
3   2021-12-04 06:58:29  2021-12-04 16:08:07  2021-12-04
4   2021-12-05 06:59:29  2021-12-05 16:07:57  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:27:20  2022-02-24 17:25:45  2022-02-24
86  2022-02-25 06:25:44  2022-02-25 17:27:01  2022-02-25
87  2022-02-26 06:24:08  2022-02-26 17:28:17  2022-02-26
88  2022-02-27 06:22:31  2022-02-27 17:29:33  2022-02-27
89  2022-02-28 06:20:54  2022-02-28 17:30:49  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:04  2022-12-01 16:08:56  2022-12-01
1   2022-12-02 06:56:09  2022-12-02 16:08:38  2022-12-02
2   2022-12-03 06:57:12  2022-12-03 16:08:23  2022-12-03
3   2022-12-04 06:58:14  2022-12-04 16:08:10  2022-12-04
4   2022-12-05 06:59:15  2022-12-05 16:07:59  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:27:42  2023-02-24 17:25:26  2023-02-24
86  2023-02-25 06:26:07  2023-02-25 17:26:43  2023-02-25
87  2023-02-26 06:24:32  2023-02-26 17:27:59  2023-02-26
88  2023-02-27 06:22:55  2023-02-27 17:29:15  2023-02-27
89  2023-02-28 06:21:17  2023-02-28 17:30:31  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PSM,2020-12-05 18:56:00,2.5400,-RASN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
1,PSM,2020-12-05 19:56:00,1.7800,-RASN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
2,PSM,2020-12-05 20:56:00,1.7800,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
3,PSM,2020-12-05 21:56:00,3.0500,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
4,PSM,2020-12-05 22:56:00,3.0500,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
...,...,...,...,...,...,...,...
176,PSM,2021-02-22 19:56:00,0.0001,-SN BR,2021-02-22,2021-02-22 06:30:05,2021-02-22 17:23:29
177,PSM,2021-02-27 12:56:00,0.0001,-SN,2021-02-27,2021-02-27 06:22:08,2021-02-27 17:29:52
178,PSM,2021-02-27 13:56:00,1.0200,-SN BR,2021-02-27,2021-02-27 06:22:08,2021-02-27 17:29:52
179,PSM,2021-02-27 14:56:00,1.2700,-SN BR,2021-02-27,2021-02-27 06:22:08,2021-02-27 17:29:52


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PSM,2021-12-02 09:56:00,0.0001,-RASN,2021-12-02,2021-12-02 06:56:24,2021-12-02 16:08:34
1,PSM,2021-12-02 10:56:00,0.0001,-RASN,2021-12-02,2021-12-02 06:56:24,2021-12-02 16:08:34
2,PSM,2021-12-08 13:56:00,0.0001,-SN,2021-12-08,2021-12-08 07:02:22,2021-12-08 16:07:43
3,PSM,2021-12-08 14:56:00,0.0001,-SN,2021-12-08,2021-12-08 07:02:22,2021-12-08 16:07:43
4,PSM,2021-12-09 06:56:00,0.0001,-SN,2021-12-09,2021-12-09 07:03:16,2021-12-09 16:07:43
...,...,...,...,...,...,...,...
135,PSM,2022-02-25 21:56:00,0.0001,-SN,2022-02-25,2022-02-25 06:25:44,2022-02-25 17:27:01
136,PSM,2022-02-25 22:56:00,0.2500,-SN,2022-02-25,2022-02-25 06:25:44,2022-02-25 17:27:01
137,PSM,2022-02-25 23:56:00,0.2500,-SN,2022-02-25,2022-02-25 06:25:44,2022-02-25 17:27:01
138,PSM,2022-02-26 00:56:00,0.2500,-SN,2022-02-26,2022-02-26 06:24:08,2022-02-26 17:28:17


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PSM,2022-12-11 21:55:00,0.0001,-SN,2022-12-11,2022-12-11 07:04:48,2022-12-11 16:07:49
1,PSM,2022-12-11 22:55:00,0.0001,-SN,2022-12-11,2022-12-11 07:04:48,2022-12-11 16:07:49
2,PSM,2022-12-12 00:55:00,0.0001,-SN,2022-12-12,2022-12-12 07:05:37,2022-12-12 16:07:56
3,PSM,2022-12-12 01:55:00,0.0001,-SN,2022-12-12,2022-12-12 07:05:37,2022-12-12 16:07:56
4,PSM,2022-12-12 02:55:00,0.0001,-SN,2022-12-12,2022-12-12 07:05:37,2022-12-12 16:07:56
...,...,...,...,...,...,...,...
185,PSM,2023-02-26 18:55:00,0.0001,-SN BR,2023-02-26,2023-02-26 06:24:32,2023-02-26 17:27:59
186,PSM,2023-02-26 19:55:00,0.0001,-SN,2023-02-26,2023-02-26 06:24:32,2023-02-26 17:27:59
187,PSM,2023-02-26 20:55:00,0.0001,-SN,2023-02-26,2023-02-26 06:24:32,2023-02-26 17:27:59
188,PSM,2023-02-27 00:55:00,0.0001,-SN BR,2023-02-27,2023-02-27 06:22:55,2023-02-27 17:29:15


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
20,PSM,2020-12-17 07:56:00,0.7600,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
21,PSM,2020-12-17 08:56:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
22,PSM,2020-12-17 09:56:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
23,PSM,2020-12-17 10:56:00,0.7600,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
24,PSM,2020-12-17 11:56:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
25,PSM,2020-12-17 12:56:00,0.0001,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
26,PSM,2020-12-17 13:56:00,0.0001,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
27,PSM,2020-12-17 14:56:00,0.2500,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
28,PSM,2020-12-17 15:56:00,0.2500,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18
42,PSM,2021-01-02 07:56:00,1.0200,-RAPL BR,2021-01-02,2021-01-02 07:14:59,2021-01-02 16:19:44


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
21,PSM,2020-12-17 08:56:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
22,PSM,2020-12-17 09:56:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
23,PSM,2020-12-17 10:56:00,0.7600,SN FZFG,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
24,PSM,2020-12-17 11:56:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
25,PSM,2020-12-17 12:56:00,0.0001,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
26,PSM,2020-12-17 13:56:00,0.0001,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
27,PSM,2020-12-17 14:56:00,0.2500,-SN,2020-12-17,2020-12-17 07:09:36,2020-12-17 16:09:18,2020-12-17 08:09:36,2020-12-17 15:09:18
46,PSM,2021-01-05 11:56:00,0.0001,-SN,2021-01-05,2021-01-05 07:14:52,2021-01-05 16:22:37,2021-01-05 08:14:52,2021-01-05 15:22:37
47,PSM,2021-01-05 12:56:00,0.0001,-SN,2021-01-05,2021-01-05 07:14:52,2021-01-05 16:22:37,2021-01-05 08:14:52,2021-01-05 15:22:37
48,PSM,2021-01-05 13:56:00,0.0001,-SN BR,2021-01-05,2021-01-05 07:14:52,2021-01-05 16:22:37,2021-01-05 08:14:52,2021-01-05 15:22:37


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.2158085106382978
2021-2022: 0.4457413793103449
2022-2023: 0.22829411764705884


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PSM,2020-12-05 18:56:00,2.5400,-RASN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
1,PSM,2020-12-05 19:56:00,1.7800,-RASN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
2,PSM,2020-12-05 20:56:00,1.7800,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
3,PSM,2020-12-05 21:56:00,3.0500,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
4,PSM,2020-12-05 22:56:00,3.0500,-SN BR,2020-12-05,2020-12-05 06:59:44,2020-12-05 16:07:56
...,...,...,...,...,...,...,...
166,PSM,2021-02-20 02:56:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:33:09,2021-02-20 17:20:55
167,PSM,2021-02-20 03:56:00,0.0001,-SN,2021-02-20,2021-02-20 06:33:09,2021-02-20 17:20:55
168,PSM,2021-02-20 04:56:00,0.0001,-SN,2021-02-20,2021-02-20 06:33:09,2021-02-20 17:20:55
175,PSM,2021-02-22 18:56:00,0.0001,-SN,2021-02-22,2021-02-22 06:30:05,2021-02-22 17:23:29


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_39356\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.49955409836065573
2021-2022: 0.5273947368421052
2022-2023: 0.2774188679245283


In [65]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NH'

data.to_csv('SO.psm.csv', index=False)